In [ ]:
%pip install langchain langchain-community fastapi uvicorn requests duckduckgo-search aiohttp nest_asyncio psutil tenacity yandexcloud -q

In [ ]:
# в ячейке ноутбука
%pip install -U nbformat


In [ ]:
%pip install matplotlib seaborn pandas

In [ ]:
%pip install ddgs

In [ ]:
%pip install streamlit

In [ ]:
# %cd ..

In [ ]:
# from mini_langsmith import (
#     Tracer, attach_logger, parse_logs_to_df,
#     show_span_tree, summarize, spans_to_dataframe, export_spans
# )

# Import

In [1]:
import logging
import time
import threading
import uvicorn
import requests
import aiohttp
import asyncio
import nest_asyncio
import psutil
import socket
import json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.callbacks import StdOutCallbackHandler
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.llms import YandexGPT
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import re
from pydantic import BaseModel, Field, ValidationError
import uuid
import aiohttp
# from prometheus_client import Counter, Histogram


In [2]:
# Apply nest_asyncio for Jupyter compatibility
nest_asyncio.apply()

In [3]:
# Configure logging
logging.basicConfig(
    filename='system.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [4]:
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()
FODLER_ID =  os.getenv("FOLDER_ID")
API_KEY =  os.getenv("API_KEY")


# Логи и трассировка

In [ ]:

import sys

In [7]:
%cd ..

d:\ai_product_hack


In [8]:
sys.path.append(str("multiagent_tracer/"))

In [ ]:

from multiagent_tracer import (
    MultiAgentTracer, AgentType,
    trace_agent, trace_async_agent,
)


In [ ]:

from multiagent_tracer import AgentType


In [11]:
tracer = MultiAgentTracer(
    session_id="notebook_session",
    log_file=None,                 # можно указать путь к файлу .log
    enable_real_time_viz=False,    # True — если хотите обновление графа на лету
)
print(f"Tracer session_id = {tracer.session_id}")

Tracer session_id = notebook_session


In [ ]:
# file: custom_trace.py
import logging, sys, json
from logging.handlers import RotatingFileHandler
import time
from starlette.middleware.base import BaseHTTPMiddleware
import uuid
from typing import Callable, Optional
from langchain.callbacks.base import BaseCallbackHandler
from starlette.responses import Response
from fastapi import Request


In [ ]:
# file: custom_trace.py
from contextlib import contextmanager

@contextmanager
def trace_block(agent_name: str,
                agent_type: AgentType = AgentType.CUSTOM,
                parent_event_id: str | None = None,
                **data):
    eid = tracer.start_trace(
        agent_name=agent_name,
        agent_type=agent_type,
        data=data,
        parent_event_id=parent_event_id
    )
    try:
        yield eid
    except Exception as e:
        # фикс: логируем и ЗДЕСЬ же безопасно перекидываем исключение
        tracer.log_error(agent_name, e, parent_event_id=eid)
        tracer.end_trace(eid, {"error": str(e)}, success=False)
        raise
    else:
        # если исключения не было — закрываем с success=True
        tracer.end_trace(eid, {"success": True}, success=True)

In [ ]:
# file: custom_trace.py
MAX_LOG_CHARS = 2000

def _safe_text(b: bytes) -> str:
    try:
        return b.decode("utf-8", errors="replace")
    except Exception:
        return "<binary>"

def _truncate(s: str, n: int = MAX_LOG_CHARS) -> str:
    return s if len(s) <= n else s[:n] + "...<truncated>"

class AccessLogMiddleware(BaseHTTPMiddleware):
    def __init__(self, app, service_name: str, log_fn: Callable[[dict], None]):
        super().__init__(app)
        self.service_name = service_name
        self.log_fn = log_fn

    async def dispatch(self, request, call_next):
        trace_id = request.headers.get("X-Request-Id", str(uuid.uuid4()))
        start_ts = time.time()

        # --- Request logging
        raw_req = await request.body()
        self.log_fn(
            service=self.service_name,
            event="request_in",
            trace_id=trace_id,
            method=request.method,
            path=str(request.url.path),
            body=_truncate(_safe_text(raw_req)),
        )

        # --- Call downstream
        resp = await call_next(request)

        # Drain body_iterator to log response body
        body = b""
        async for chunk in resp.body_iterator:
            body += chunk

        # Preserve headers & content type
        headers = dict(resp.headers)
        if "content-type" not in {k.lower(): v for k, v in headers.items()}:
            # Умная догадка: если это JSON-подобно — ставим JSON
            if body[:1] in (b"{", b"["):
                headers["content-type"] = "application/json; charset=utf-8"
            else:
                headers["content-type"] = "text/plain; charset=utf-8"

        # Rebuild response WITHOUT media_type arg, чтобы не перезаписать content-type
        new_resp = Response(
            content=body,
            status_code=resp.status_code,
            headers=headers,
        )

        # --- Response logging
        self.log_fn(
            service=self.service_name,
            event="response_out",
            trace_id=trace_id,
            status=resp.status_code,
            duration_ms=int((time.time() - start_ts) * 1000),
            body=_truncate(_safe_text(body)),
        )
        return new_resp

In [ ]:
# file: custom_trace.py
def setup_logging(log_path="orchestrator.log"):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    if not logger.handlers:
        fh = RotatingFileHandler(log_path, maxBytes=5_000_000, backupCount=3, encoding="utf-8")
        sh = logging.StreamHandler(sys.stdout)
        fmt = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
        fh.setFormatter(fmt); sh.setFormatter(fmt)
        logger.addHandler(fh); logger.addHandler(sh)

def log_json(**kwargs):
    logging.info(json.dumps(kwargs, ensure_ascii=False))

In [ ]:
# file: custom_trace.py
async def log_trace(event: str, **fields):
    fields.update({
        "ts": time.time(),
        "event": event
    })
    log_json(**fields)

In [ ]:
# file: custom_trace.py
class LoggingCallback(BaseCallbackHandler):
    def __init__(self, log_fn: Optional[Callable[..., None]] = None, log_file: str = "trace_log.log"):
        self.log_fn = log_fn
        self.log_file = log_file

    def _emit(self, **fields):
        # в файл (чтобы всегда что-то писалось)
        ts = time.strftime("%Y-%m-%d %H:%M:%S")
        line = f"[{ts}] " + " ".join(f"{k}={repr(v)[:500]}" for k, v in fields.items())
        with open(self.log_file, "a", encoding="utf-8") as f:
            f.write(line + "\n")
        # в внешний логгер, если передан
        if self.log_fn:
            try:
                self.log_fn(**fields)  # ИМЕНОВАННЫЕ аргументы!
            except Exception as e:
                logging.warning(f"LoggingCallback log_fn failed: {e}")

    # LLM события
    def on_llm_start(self, serialized, prompts, **kwargs):
        self._emit(event="llm_start", prompts=prompts)

    def on_llm_end(self, response, **kwargs):
        try:
            text = response.generations[0][0].text
        except Exception:
            text = str(response)
        self._emit(event="llm_end", text=text[:2000])

    # Цепочки
    def on_chain_start(self, serialized, inputs, **kwargs):
        self._emit(event="chain_start", inputs=inputs)

    def on_chain_end(self, outputs, **kwargs):
        self._emit(event="chain_end", outputs=str(outputs)[:2000])

    # Инструменты
    def on_tool_start(self, serialized, input_str, **kwargs):
        self._emit(event="tool_start", tool=serialized.get("name"), input=str(input_str)[:500])

    def on_tool_end(self, output, **kwargs):
        self._emit(event="tool_end", output=str(output)[:500])

# Инициализация ЛЛМ

In [ ]:
# file: agent_with_orchestration.py
# 1. Определяем Pydantic модели для всех агентов
class TaskRequest(BaseModel):
    input: str

In [ ]:
# file: agent_with_orchestration.py
# Initialize YandexGPT LLM
llm = YandexGPT(
    api_key=API_KEY,
    folder_id=FODLER_ID,
    temperature=0.2
)

In [ ]:
# file: agent_with_orchestration.py
class RouteDecision(BaseModel):
    next_agent: str = Field(pattern="^(SEARCH_AGENT|ANALYSIS_AGENT|REPORT_AGENT)$")
    reason: str
    input: str

class RouterAgent:
    def __init__(self, llm):
        self.llm = llm
        self.router_prompt = PromptTemplate.from_template(
            """You are a strict router. Choose the next agent.

            Available agents:
            - SEARCH_AGENT: use ONLY when fresh/current web info is needed or no context yet.
            - ANALYSIS_AGENT: process/summarize/structure text snippets.
            - REPORT_AGENT: produce final structured answer (last step).

            Return ONLY a JSON object (no prose, no Markdown fences).

            Input query: {query}
            Current context: {context}

            JSON schema:
            {{
            "next_agent": "SEARCH_AGENT|ANALYSIS_AGENT|REPORT_AGENT",
            "reason": "string",
            "input": "string"
            }}"""
        )

    def _extract_json(self, s: str) -> dict:
        # 1) попробуем прямой json.loads
        try:
            return json.loads(s)
        except Exception:
            pass
        # 2) выцепим самый длинный JSON-блок фигурных скобок (устойчивее, чем {[^}]*})
        candidates = []
        stack = []
        start = None
        for i, ch in enumerate(s):
            if ch == '{':
                if not stack:
                    start = i
                stack.append('{')
            elif ch == '}':
                if stack:
                    stack.pop()
                    if not stack and start is not None:
                        candidates.append(s[start:i+1])
        candidates.sort(key=len, reverse=True)
        for c in candidates:
            try:
                return json.loads(c)
            except Exception:
                continue
        raise ValueError("No valid JSON found in router output")

    def route(self, query: str, context: str = "No context yet"):
        try:
            chain = self.router_prompt | self.llm
            raw = chain.invoke({"query": query, "context": context})
            data = self._extract_json(raw)
            decision = RouteDecision(**data)  # валидация
            return decision.model_dump()
        except Exception as e:
            logging.error(f"Router LLM failed: {e}. Fallback engaged.")
            return self._smart_fallback(query, context)

    def _smart_fallback(self, query, context):
        q = query.lower()
        c = context.lower()

        # Если уже есть выход поиска — идём в анализ
        if "search_agent output" in c or "найдено" in c or "результат поиска" in c:
            return {
                "next_agent": "ANALYSIS_AGENT",
                "reason": "Post-search analysis step",
                "input": context
            }
        # Если уже есть анализ — финализируем отчёт
        if "analysis_agent output" in c or "анализ" in c or "summary" in c:
            return {
                "next_agent": "REPORT_AGENT",
                "reason": "Finalize into report",
                "input": context
            }
        # Триггеры «нужна свежесть»
        if any(t in q for t in ["нов", "свеж", "сегодня", "вчера", "202", "цена", "курс", "последн", "latest", "today", "news"]):
            return {"next_agent": "SEARCH_AGENT", "reason": "Fresh info likely needed", "input": query}
        # По умолчанию — анализ
        return {"next_agent": "ANALYSIS_AGENT", "reason": "Direct analysis", "input": query}


In [ ]:
# file: agent_with_orchestration.py
TRACE_SESSION = "X-Trace-Session"
TRACE_PARENT = "X-Parent-Event"

class TraceHeadersMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        sid = request.headers.get(TRACE_SESSION)
        if sid:
            try:
                tracer.session_id = sid
            except Exception:
                pass
        response = await call_next(request)
        return response
    


In [ ]:
# file: agent_with_orchestration.py
# 2. ПОИСКОВЫЙ АГЕНТ (отдельный сервис)
search_app = FastAPI(title="search-agent")
search_app.add_middleware(AccessLogMiddleware, service_name="search-agent", log_fn=log_json)

duck = DuckDuckGoSearchRun()

translate_prompt = PromptTemplate.from_template(
    "Translate to English for web search. Output ONLY the translation, no quotes, no comments:\n{data}"
)

@search_app.post("/execute")
async def execute_search(task: TaskRequest, request: Request):
    parent = request.headers.get("X-Parent-Event")
    root_eid = tracer.start_trace(
        agent_name="search-agent/execute",
        agent_type=AgentType.SEARCH,
        data={
            "trace_headers_present": bool(parent),
            "input_preview": (task.input or "")[:200]
        },
        parent_event_id=parent
    )

    cb = LoggingCallback(log_fn=log_json)
    try:
        # 1) translate
        with trace_block("translate", AgentType.TOOL, parent_event_id=root_eid, original_query=task.input) as trans_eid:
            model_name = getattr(llm, "model_name", type(llm).__name__)
            tool_eid = tracer.log_tool_start(
                agent_name="translate",
                tool_name=str(model_name),
                tool_input={"data_preview": (task.input or "")[:200]},
                parent_event_id=trans_eid
            )
            try:
                english_query = (translate_prompt | llm).invoke({"data": task.input}, config={"callbacks":[cb]}).strip()
                if english_query.startswith('"') and english_query.endswith('"'):
                    english_query = english_query[1:-1].strip()
                tracer.log_tool_end(
                    tool_eid,
                    tool_output={"translation_preview": english_query[:400]},
                    success=True
                )
            except Exception as e:
                tracer.log_tool_end(tool_eid, tool_output={"error": str(e)[:400]}, success=False)
                tracer.log_error("translate", e, parent_event_id=trans_eid)
                tracer.end_trace(root_eid, {"error": str(e)}, success=False)
                raise HTTPException(status_code=500, detail=str(e))

        # 2) web_search
        with trace_block("web_search", AgentType.SEARCH, parent_event_id=root_eid, query=english_query) as search_eid:
            tool_eid = tracer.log_tool_start(
                agent_name="web_search",
                tool_name="DuckDuckGoSearchRun",
                tool_input={"query": english_query},
                parent_event_id=search_eid
            )
            try:
                result = duck.run(english_query)
                tracer.log_tool_end(
                    tool_eid,
                    tool_output={"result_preview": str(result)[:400]},
                    success=True
                )
            except Exception as e:
                tracer.log_tool_end(tool_eid, tool_output={"error": str(e)[:400]}, success=False)
                tracer.log_error("web_search", e, parent_event_id=search_eid)
                tracer.end_trace(root_eid, {"error": str(e)}, success=False)
                raise HTTPException(status_code=500, detail=str(e))

        tracer.end_trace(root_eid, {"success": True}, success=True)
        return {"result": result, "agent": "SEARCH_AGENT", "translated_query": english_query}

    except HTTPException:
        raise
    except Exception as e:
        tracer.log_error("search/execute", e, parent_event_id=root_eid)
        tracer.end_trace(root_eid, {"error": str(e)}, success=False)
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        tracer.export_traces("search_agent.json", format="json")


In [ ]:
# file: agent_with_orchestration.py
# 3. АГЕНТ-АНАЛИТИК (отдельный сервис)
analysis_app = FastAPI()
analysis_app.add_middleware(AccessLogMiddleware, service_name="analysis-agent", log_fn=log_json)

@analysis_app.post("/execute")
async def execute_analysis(task: TaskRequest, request: Request):
    parent = request.headers.get("X-Parent-Event")
    root_eid = tracer.start_trace(
        agent_name="analysis-agent/execute",
        agent_type=AgentType.ANALYSIS,
        data={
            "trace_headers_present": bool(parent),
            "input_preview": (task.input or "")[:200]
        },
        parent_event_id=parent
    )

    cb = LoggingCallback(log_fn=log_json)
    try:
        # prepare_prompt
        with trace_block(
            "prepare_prompt",
            AgentType.CUSTOM,
            parent_event_id=root_eid,
            template="Analyze this information and extract key insights in Russian"
        ) as prep_eid:
            prompt = PromptTemplate.from_template("""
            Analyze this information and extract key insights in Russian:

            {data}

            Provide comprehensive analysis in Russian.
            """)
            chain = prompt | llm

        # llm_inference
        with trace_block(
            "llm_inference",
            AgentType.ANALYSIS,
            parent_event_id=root_eid,  # <-- важное: родитель — текущий блок, а не prep_eid
            tokens_estimate=len((task.input or ""))  # просто пример метадаты
        ) as infer_eid:
            model_name = getattr(llm, "model_name", type(llm).__name__)
            tool_eid = tracer.log_tool_start(
                agent_name="llm_inference",              # <-- это имя пойдёт в граф (будет "llm_inference::MODEL")
                tool_name=str(model_name),
                tool_input={"data_preview": (task.input or "")[:200]},
                parent_event_id=infer_eid                # <-- правильный parent
            )
            try:
                result = chain.invoke({"data": task.input}, config={"callbacks":[cb]})
                tracer.log_tool_end(
                    tool_eid,
                    tool_output={"result_preview": (str(result) if result is not None else "")[:400]},
                    success=True
                )
            except Exception as e:
                tracer.log_tool_end(
                    tool_eid,
                    tool_output={"error": str(e)[:400]},
                    success=False
                )
                tracer.log_error("llm_inference", e, parent_event_id=infer_eid)
                tracer.end_trace(root_eid, {"error": str(e)}, success=False)
                raise HTTPException(status_code=500, detail=str(e))

        tracer.end_trace(root_eid, {"success": True}, success=True)
        return {"result": result, "agent": "ANALYSIS_AGENT"}

    except HTTPException:
        # уже залогировано выше
        raise
    except Exception as e:
        tracer.log_error("analysis-agent/execute", e, parent_event_id=root_eid)
        tracer.end_trace(root_eid, {"error": str(e)}, success=False)
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        tracer.export_traces("analysis_agent.json", format="json")

In [ ]:
# file: agent_with_orchestration.py
# 4. АГЕНТ-ОТЧЕТ (отдельный сервис)
report_app = FastAPI()
report_app.add_middleware(AccessLogMiddleware, service_name="report-agent", log_fn=log_json)

@report_app.post("/execute")
async def execute_report(task: TaskRequest, request: Request):
    parent = request.headers.get("X-Parent-Event")
    root_eid = tracer.start_trace(
        agent_name="report-agent/execute",
        agent_type=AgentType.REPORT,
        data={
            "trace_headers_present": bool(parent),
            "input_preview": (task.input or "")[:300]
        },
        parent_event_id=parent
    )
    try:
        cb = LoggingCallback(log_fn=log_json)

        # подготовка промпта
        with trace_block("prepare_prompt", AgentType.CUSTOM, parent_event_id=root_eid) as prep_eid:
            prompt = PromptTemplate.from_template(
                """Generate comprehensive final answer in Russian based on this information:

                {data}

                Answer in clear, structured Russian."""
            )
            chain = prompt | llm

        # инференс LLM (как инструмент: start -> invoke -> end)
        with trace_block("llm_inference", AgentType.REPORT, parent_event_id=prep_eid) as infer_eid:
            model_name = getattr(llm, "model_name", type(llm).__name__)
            tool_eid = tracer.log_tool_start(
                agent_name="llm_inference",
                tool_name=str(model_name),
                tool_input={"data_preview": (task.input or "")[:200]},
                parent_event_id=infer_eid
            )
            try:
                result = chain.invoke({"data": task.input}, config={"callbacks": [cb]})
                tracer.log_tool_end(
                    tool_eid,
                    tool_output={"result_preview": (str(result) if result is not None else "")[:400]},
                    success=True
                )
            except Exception as e:
                tracer.log_tool_end(
                    tool_eid,
                    tool_output={"error": str(e)[:400]},
                    success=False
                )
                raise

        tracer.end_trace(root_eid, {"success": True}, success=True)
        return {"result": result, "agent": "REPORT_AGENT"}

    except Exception as e:
        tracer.log_error("report-agent/execute", e, parent_event_id=root_eid)
        tracer.end_trace(root_eid, {"error": str(e)}, success=False)
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        tracer.export_traces("report_agent.json", format="json")
      


In [ ]:
# file: agent_with_orchestration.py
# 5. ГЛАВНЫЙ ОРКЕСТРАТОР (управляет всей системой)
orchestrator_app = FastAPI()
orchestrator_app.add_middleware(AccessLogMiddleware, service_name="orchestrator", log_fn=log_json)


router = RouterAgent(llm)

# Маппинг агентов на их URL
AGENT_URLS = {
    "SEARCH_AGENT": "http://localhost:8001/execute",
    "ANALYSIS_AGENT": "http://localhost:8002/execute", 
    "REPORT_AGENT": "http://localhost:8003/execute"
}

In [ ]:
# file: agent_with_orchestration.py
for _app in [orchestrator_app, search_app, analysis_app, report_app]:
    try:
        _app.add_middleware(TraceHeadersMiddleware)
    except Exception:
        pass

In [ ]:
# file: agent_with_orchestration.py
@orchestrator_app.post("/orchestrate")
async def orchestrate(task: TaskRequest):
    request_id = str(uuid.uuid4())
    execution_chain = []
    current_data = task.input
    context = f"[{request_id}] Original query: {task.input}"
    last_agent = None
    search_runs = 0

    # корневое событие запроса оркестратора
    root_eid = tracer.start_trace(
        agent_name="orchestrator/orchestrate",
        agent_type=AgentType.ORCHESTRATOR,
        data={"request_id": request_id, "input_preview": (task.input or "")[:300]}
    )

    try:
        for step in range(6):
            # ── Роутинг шага
            with trace_block(
                agent_name="router",
                agent_type=AgentType.CLARIFICATION,
                parent_event_id=root_eid,
                step=step + 1
            ) as route_eid:

                routing = router.route(task.input if step == 0 else current_data, context)
                next_agent = routing["next_agent"]
                agent_input = routing["input"]

                # Жёсткие перила
                if last_agent == "SEARCH_AGENT" and next_agent == "SEARCH_AGENT":
                    next_agent = "ANALYSIS_AGENT"
                    agent_input = f"Analyze following search snippets:\n{current_data[:5000]}"
                if last_agent == "ANALYSIS_AGENT" and next_agent == "ANALYSIS_AGENT":
                    next_agent = "REPORT_AGENT"
                    agent_input = f"Generate report from analysis:\n{current_data[:5000]}"

                if next_agent == "SEARCH_AGENT":
                    search_runs += 1
                    if search_runs > 2:
                        next_agent = "ANALYSIS_AGENT"
                        agent_input = f"Analyze following search snippets:\n{current_data[:5000]}"

                agent_url = AGENT_URLS.get(next_agent)
                if not agent_url:
                    raise RuntimeError(f"No URL for agent {next_agent}")

                # лог «сообщения» (для диаграммы последовательности)
                tracer.log_message(
                    from_agent="orchestrator",
                    to_agent=next_agent,
                    message=f"step={step+1}, reason={routing.get('reason','')}, input_preview={agent_input[:180]}",
                    parent_event_id=route_eid
                )
                await log_trace(
                    event="route",
                    request_id=request_id,
                    step=step + 1,
                    next_agent=next_agent,
                    reason=routing.get("reason", ""),
                    agent_input_preview=agent_input[:300]
                )

            with trace_block(
                agent_name=f"http_call::{next_agent}",
                agent_type=AgentType.TOOL,
                parent_event_id=route_eid,
                url=agent_url
            ) as http_eid:

                HTTP_HEADERS = {
                    "X-Trace-Session": tracer.session_id,
                    "X-Parent-Event": http_eid,
                }

                tool_eid = tracer.log_tool_start(
                    agent_name=f"http_call::{next_agent}",   # <-- одинаковый agent_name в start и end
                    tool_name="HTTP POST",
                    tool_input={"url": agent_url, "json": {"input": agent_input[:500]}},
                    parent_event_id=http_eid
                )

                try:
                    async with aiohttp.ClientSession() as session:
                        async with session.post(agent_url, json={"input": agent_input}, headers=HTTP_HEADERS) as resp:
                            body_text = await resp.text()
                            if resp.status != 200:
                                tracer.log_tool_end(
                                    tool_eid,
                                    tool_output={"status": resp.status, "body": body_text[:400]},
                                    success=False
                                )
                                tracer.log_error(f"http_call::{next_agent}",
                                                RuntimeError(f"HTTP {resp.status}: {body_text[:400]}"),
                                                parent_event_id=http_eid)
                                raise RuntimeError(f"{next_agent} failed: {body_text}")

                            payload = await resp.json()
                            current_data = payload["result"]
                            tracer.log_tool_end(
                                tool_eid,
                                tool_output={"status": resp.status, "result_preview": str(current_data)[:400]},
                                success=True
                            )

                except Exception as e:
                    # чтобы end-тул под любой ошибкой точно был написан (если не успели выше)
                    try:
                        tracer.log_tool_end(tool_eid, tool_output={"error": str(e)[:400]}, success=False)
                    except Exception:
                        pass
                    tracer.log_error(f"http_call::{next_agent}", e, parent_event_id=http_eid)
                    raise

            # обновляем контекст и цепочку
            context += f" | {next_agent} output: {current_data[:400]}"
            execution_chain.append({
                "step": step + 1,
                "agent": next_agent,
                "input": (agent_input or "")[:250],
                "output": (current_data or "")[:250]
            })
            last_agent = next_agent

            if next_agent == "REPORT_AGENT":
                break

        # успех — закрываем корень
        tracer.end_trace(root_eid, {"success": True}, success=True)
        return {
            "final_result": current_data,
            "execution_chain": execution_chain,
            "trace_id": request_id,
            "status": "completed" if last_agent == "REPORT_AGENT" else "max_steps_reached"
        }

    except Exception as e:
        # ошибка — логируем и закрываем корень с флагом fail
        logging.exception(f"[{request_id}] Orchestration error: {e}")
        tracer.log_error("orchestrator/orchestrate", e, parent_event_id=root_eid)
        tracer.end_trace(root_eid, {"error": str(e)}, success=False)
        return {
            "final_result": f"Error: {e}",
            "execution_chain": execution_chain,
            "trace_id": request_id,
            "status": "error"
        }
    finally:
        tracer.export_traces("orchestrator_trace.json", format="json")


In [ ]:
# file: agent_with_orchestration.py
@orchestrator_app.post("/test_router")
async def test_router(task: TaskRequest):
    """Тестовый эндпоинт для проверки работы роутера"""
    # Корневой эвент запроса
    root_eid = tracer.start_trace(
        agent_name="orchestrator/test_router",
        agent_type=AgentType.ORCHESTRATOR,
        data={"input_preview": (task.input or "")[:300]}
    )
    try:
        # Отдельный блок для принятия решения роутером
        with trace_block(
            agent_name="router",
            agent_type=AgentType.CLARIFICATION,
            parent_event_id=root_eid
        ) as route_eid:
            routing = router.route(task.input)

            # Лог «сообщения» для диаграммы последовательности
            tracer.log_message(
                from_agent="orchestrator",
                to_agent=routing.get("next_agent", "UNKNOWN"),
                message=f"router_decision: {routing}",
                parent_event_id=route_eid
            )

        response = {
            "input": task.input,
            "routing_decision": routing,
            "agent_url": AGENT_URLS.get(routing["next_agent"])
        }

        # Успешное закрытие корневого эвента
        tracer.end_trace(root_eid, {"success": True}, success=True)
        return response

    except Exception as e:
        # Ошибочное закрытие + запись ошибки
        tracer.log_error("orchestrator/test_router", e, parent_event_id=root_eid)
        tracer.end_trace(root_eid, {"error": str(e)}, success=False)
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        tracer.export_traces("orchestrator_test_router_trace.json", format="json")

In [ ]:
# file: agent_with_orchestration.py
# Health checks
@orchestrator_app.get("/health")
async def health():
    return {"status": "healthy", "service": "orchestrator"}

@search_app.get("/health")
async def search_health():
    return {"status": "healthy", "service": "search_agent"}

@analysis_app.get("/health")
async def analysis_health():
    return {"status": "healthy", "service": "analysis_agent"}

@report_app.get("/health")
async def report_health():
    return {"status": "healthy", "service": "report_agent"}

# Подтягиваем порты апишек

In [30]:
# file: agent_with_orchestration.py
def run_server(app, port, log_file):
    """Запускает FastAPI сервер на указанном порту"""
    logging.basicConfig(
        filename=log_file, 
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    try:
        logging.info(f"Starting server on port {port}")
        uvicorn.run(
            app, 
            host="0.0.0.0", 
            port=port, 
            log_level="info",
            access_log=False  # Убираем лишние логи
        )
    except Exception as e:
        logging.error(f"Server failed to start on port {port}: {str(e)}")
        raise

# Освобождаем порты перед запуском
def free_port(port):
    """Освобождает порт если он занят"""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('localhost', port)) == 0:
                # Порт занят, пытаемся освободить
                for proc in psutil.process_iter(['pid', 'name']):
                    try:
                        for conn in proc.net_connections():
                            if conn.laddr.port == port:
                                proc.terminate()
                                proc.wait(timeout=2)
                                logging.info(f"Freed port {port} from process {proc.pid}")
                    except (psutil.NoSuchProcess, psutil.AccessDenied):
                        pass
    except Exception as e:
        logging.warning(f"Could not free port {port}: {str(e)}")

# Освобождаем все нужные порты
for port in [8000, 8001, 8002, 8003]:
    free_port(port)
    time.sleep(1)

# Запускаем все сервисы
services = [
    (orchestrator_app, 8000, "orchestrator"),
    (search_app, 8001, "search_agent"), 
    (analysis_app, 8002, "analysis_agent"),
    (report_app, 8003, "report_agent")
]

for app, port, name in services:
    threading.Thread(
        target=run_server, 
        args=(app, port, f"{name}.log"),
        daemon=True  # Демонизируем потоки
    ).start()
    time.sleep(2)  # Пауза между запуском сервисов

logging.info("All agents started successfully!")
print("Multi-agent system is running on ports 8000-8003")

INFO:     Started server process [13484]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started server process [13484]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
INFO:     Started server process [13484]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)
INFO:     Started server process [13484]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8003 (Press CTRL+C to quit)


Multi-agent system is running on ports 8000-8003


In [31]:
# file: agent_with_orchestration.py
# System Status Monitor
class SystemMonitor:
    def __init__(self):
        self.agent_status = {
            "orchestrator": False,
            "search_agent": False,
            "analysis_agent": False,
            "report_agent": False
        }
    
    async def check_all_services(self):
        for service, port in [
            ("orchestrator", 8000),
            ("search_agent", 8001),
            ("analysis_agent", 8002),
            ("report_agent", 8003)
        ]:
            try:
                async with aiohttp.ClientSession() as session:
                    async with session.get(f"http://localhost:{port}/health", timeout=5) as response:
                        if response.status == 200:
                            self.agent_status[service] = True
                            logging.info(f"{service} is healthy")
                        else:
                            self.agent_status[service] = False
                            logging.warning(f"{service} returned status {response.status}")
            except Exception as e:
                self.agent_status[service] = False
                logging.error(f"{service} health check failed: {str(e)}")
    
    def all_services_healthy(self):
        return all(self.agent_status.values())

    def get_status_report(self):
        return {
            "system_status": "healthy" if self.all_services_healthy() else "degraded",
            "agent_status": self.agent_status
        }

# Initialize system monitor
system_monitor = SystemMonitor()

# Run initial health check
await system_monitor.check_all_services()
print("System Status:", system_monitor.get_status_report())

System Status: {'system_status': 'healthy', 'agent_status': {'orchestrator': True, 'search_agent': True, 'analysis_agent': True, 'report_agent': True}}


# Тест промпт


In [32]:
async def debug_test():
    """Тест с детальным логированием"""
    test_query = "Какая сегодня погода в Москве?"
    
    async with aiohttp.ClientSession() as session:
        async with session.post(
            "http://localhost:8000/orchestrate",
            json={"input": test_query}
        ) as response:
            result = await response.json()
            
            print(f"\nQuery: {test_query}")
            print(f"Status: {result['status']}")
            print(f"Steps: {len(result['execution_chain'])}")
            
            for step in result['execution_chain']:
                print(f"\nStep {step['step']} - {step['agent']}:")
                print(f"   Input: {step['input'][:80]}...")
                print(f"   Output: {step['output'][:80]}...")
            
            print(f"\nFinal result: {result['final_result'][:200]}...")

# Запускаем тест
asyncio.run(debug_test())


Query: Какая сегодня погода в Москве?
Status: max_steps_reached
Steps: 6

Step 1 - SEARCH_AGENT:
   Input: Какая сегодня погода в Москве?...
   Output: 3 days ago - Due to the significant variation in temperature between the winter ...

Step 2 - ANALYSIS_AGENT:
   Input: Analyze following search snippets:
3 days ago - Due to the significant variation...
   Output: Из представленных поисковых сниппетов можно извлечь следующие ключевые моменты:
...

Step 3 - SEARCH_AGENT:
   Input: Какая сегодня погода в Москве?...
   Output: 4 uur geleden — Current weather in Moscow and forecast for today, tomorrow, and ...

Step 4 - ANALYSIS_AGENT:
   Input: Какая сегодня погода в Москве?...
   Output: Запрос «Какая сегодня погода в Москве?» направлен на получение информации о пого...

Step 5 - ANALYSIS_AGENT:
   Input: Analyze following search snippets:
Запрос «Какая сегодня погода в Москве?» напра...
   Output: Запрос «Какая сегодня погода в Москве?» направлен на получение информации о пого...

Step

# Работа со streamlit

In [33]:
import sys, subprocess, socket, time, os
import matplotlib.pyplot as plt

PORT = 8501  # при необходимости измените порт
APP  = "streamlit_app.py"  # путь к вашему app

# Если уже запущен на этом порту — выберите другой
def port_open(port):
    with socket.socket() as s:
        s.settimeout(0.2)
        return s.connect_ex(("127.0.0.1", port)) == 0

if port_open(PORT):
    print(f"Порт {PORT} уже занят. Поменяйте PORT или остановите другой процесс.")
else:
    # Запускаем streamlit тем же интерпретатором, что и Jupyter
    cmd = [
        sys.executable, "-m", "streamlit", "run", APP,
        "--server.headless", "true",
        "--server.port", str(PORT)
    ]
    # stdout/stderr можно писать в файл, чтобы не засорять вывод
    log_file = open("streamlit.log", "w", encoding="utf-8")
    proc = subprocess.Popen(cmd, stdout=log_file, stderr=subprocess.STDOUT)

    # Подождём, пока сервер поднимется
    for _ in range(60):
        if port_open(PORT):
            break
        time.sleep(0.5)

    if port_open(PORT):
        print(f"Streamlit запущен: http://localhost:{PORT}")
        print("Логи пишутся в streamlit.log")
    else:
        print("Не удалось запустить Streamlit (проверьте streamlit.log)")


Streamlit запущен: http://localhost:8501
Логи пишутся в streamlit.log


# Освобождение порта

In [48]:
import psutil, os

def kill_on_port(port):
    for p in psutil.process_iter(attrs=["pid","name","cmdline"]):
        try:
            cmd = " ".join(p.info.get("cmdline") or [])
            if "streamlit" in cmd and f"--server.port {port}" in cmd:
                print("Завершаю PID", p.pid)
                p.terminate()
        except (psutil.AccessDenied, psutil.NoSuchProcess):
            pass

kill_on_port(PORT)

Завершаю PID 16340
Завершаю PID 16412
